### Kaggle

In [39]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [40]:
creds = '{"username":"jonathasdossantos","key":"8c63c293837954ffe2aff9ef10e07867"}'

In [41]:
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

Podemos agora baixar os datasets do Kaggle

In [42]:
path = Path('/kaggle/input/us-patent-phrase-to-phrase-matching')

In [43]:
if not iskaggle and not path.exists():
    import zipfile,kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

In [44]:
!ls {path}

sample_submission.csv  test.csv  train.csv


In [45]:
import pandas as pd

Vamos armazenar o Path de nossos dados em uma variável:

In [46]:
df = pd.read_csv(path/'train.csv')

In [47]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [48]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [49]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [50]:
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

## Tokenização

In [51]:
!pip install datasets

In [52]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)

In [53]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

### O pacote transformers nos dará os recursos relacionados à tokenização

In [54]:
!pip -q install transformers

In [55]:
model_nm = 'microsoft/deberta-v3-small'

In [56]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [57]:
def tok_func(x): return tokz(x["input"])

Usando`map`, podemos aplicar a função de tokenização que criamos acima em cada umas das linhas:

In [58]:
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

In [59]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

## Set's de validação e de teste

No diretório que baixamos já existe um arquivo com o set de teste, como podemos ver abaixo:

In [60]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


### Set de validação

In [61]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

### Set de teste

In [62]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

## Metricas e correlações

In [63]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing(as_frame=True)
housing = housing['data'].join(housing['target']).sample(1000, random_state=52)
housing.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
7506,3.0550,37.0,5.152778,1.048611,729.0,5.062500,33.92,-118.28,1.054
4720,3.0862,35.0,4.697897,1.055449,1159.0,2.216061,34.05,-118.37,3.453
12888,2.5556,24.0,4.864905,1.129222,1631.0,2.395007,38.66,-121.35,1.057
13344,3.0057,32.0,4.212687,0.936567,1378.0,5.141791,34.05,-117.64,0.969
7173,1.9083,42.0,3.888554,1.039157,1535.0,4.623494,34.05,-118.19,1.192


We can see all the correlation coefficients for every combination of columns in this dataset by calling `np.corrcoef`:

In [64]:
import numpy as np
np.set_printoptions(precision=2, suppress=True)

In [65]:
def corr(x,y): return np.corrcoef(x,y)[0][1]

corr(housing.MedInc, housing.MedHouseVal)

0.6760250732906

In [66]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

## Trainando o modelo

Para treinar o modelo, precisaremos de outros recursos do pacote transformers.

In [67]:
from transformers import TrainingArguments,Trainer

In [68]:
bs = 128
epochs = 4

Vamos definir agora a variável responsável pela taxa de aprendizado (learning rate):

In [69]:
lr = 8e-5

In [70]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [71]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

Agora podemos treinar nosso modelo.

In [73]:
trainer.train();

The following columns in the training set  don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, target, anchor, context, id.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27354
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 856
/opt/conda/lib/python3.7/site-packages/transformers/trainer.py:1410: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-fin

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.025295,0.800815
2,No log,0.021321,0.826213
3,0.036200,0.021597,0.832466
4,0.036200,0.022752,0.833663


The following columns in the evaluation set  don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, target, anchor, context, id.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 256
The following columns in the evaluation set  don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, target, anchor, context, id.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 256
Saving model checkpoint to outputs/checkpoint-500
Configuration saved in outputs/checkpoint-500/config.json
Model weights saved in outputs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in outputs/checkpoint-500/tokenizer_config.json
Special tokens file saved in outputs/checkpoint-500/special_tokens_map.json
added tokens file saved in outputs/checkpoint-500/added_tokens.json
The following columns in the evaluation set  don't have a corresponding argument in `Deb

In [74]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

The following columns in the test set  don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, target, anchor, context, id.
***** Running Prediction *****
  Num examples = 36
  Batch size = 256


array([[ 0.46],
       [ 0.64],
       [ 0.62],
       [ 0.36],
       [-0.05],
       [ 0.5 ],
       [ 0.55],
       [ 0.  ],
       [ 0.25],
       [ 1.12],
       [ 0.21],
       [ 0.24],
       [ 0.74],
       [ 0.8 ],
       [ 0.76],
       [ 0.41],
       [ 0.35],
       [ 0.01],
       [ 0.75],
       [ 0.38],
       [ 0.41],
       [ 0.22],
       [ 0.16],
       [ 0.27],
       [ 0.57],
       [-0.02],
       [-0.04],
       [-0.02],
       [-0.03],
       [ 0.53],
       [ 0.3 ],
       [ 0.03],
       [ 0.74],
       [ 0.53],
       [ 0.44],
       [ 0.26]])

Se você obsevar, algumas previsões possuem valores menores que 0 ou maiores que 1. Para consertar isso, usemos a função clip:

In [80]:
preds = np.clip(preds, 0, 1)

In [81]:
preds

array([[0.46],
       [0.64],
       [0.62],
       [0.36],
       [0.  ],
       [0.5 ],
       [0.55],
       [0.  ],
       [0.25],
       [1.  ],
       [0.21],
       [0.24],
       [0.74],
       [0.8 ],
       [0.76],
       [0.41],
       [0.35],
       [0.01],
       [0.75],
       [0.38],
       [0.41],
       [0.22],
       [0.16],
       [0.27],
       [0.57],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.53],
       [0.3 ],
       [0.03],
       [0.74],
       [0.53],
       [0.44],
       [0.26]])

Pronto!!